In [1]:
import os
from os.path import join
import pandas as pd
from pathlib import Path
import sys

project_root = Path('../..')
sys.path.append(os.path.abspath(project_root))
from notebooks.utils import init_data_dir  # noqa

# from notebooks.benchmarking import benchmark_profiles  # noqa

init_data_dir(project_root)

preprocess_path = join(project_root, Path('data/preprocess'))
outputs_path = join(project_root, 'outputs')

# Progress Report 2

Much of the data preprocessing code was refactored so that different datasets could be quickly trained/evaluated on and so that the amount of sentences in a text group could be quickly modified. Benchmarks were computed for each of the profile methods using the new preprocessing functionality. Plans have been made to create a detailed report screen where the sentences that are most suspicious are highlighted.

## Benchmarks

The dataset was split into groups of 50 sentences each, with the author and the ID of the text attached to each group. Each of the profiles were tested on 10 samples of 30 authors (profile/benchmark code is very slow right now and needs to be optimized so that we can more easily test on larger sets). Out of each set of 30 authors, one author was chosen to be the "profile" author, and the others were chosen to be suspects. Out of the texts for the profile author, one was chosen to be a suspect and the rest were fed into the profile. The profile was then tested on all the suspects, and the true/false positives and negatives were collected. We score the profiles based on sensitivity, specificity, balanced accuracy, and precision.

POSPCA is performing at chance accuracy (but this seems to be due to a bug that causes it to flag everything). The heuristics are performing the best at ~75% balanced accuracy.

In [2]:
benchmark_results = pd.read_hdf(join(outputs_path,
                                     'bawe_train_benchmarks.hdf5'))

benchmark_results

,balanced accuracy,specificity,sensitivity (recall),precision
pospca,0.500000,0.0,1.000000,1.00000
heuristics,0.757503,0.7,0.815005,0.99125


## Explainability and the Detailed Report Screen

Our current idea for model explainability is to architecture each profile to work on a sentence level and then use the average of many sentence vectors to profile. This way we can analyze the difference between each sentence in a suspect text compared to the profile. If a sentence is very far from the profile, we can highlight that sentence based on how far it is, indicating that this sentence is part of the reason that the suspect text was flagged. For a profile that is more interpretable, such as the heuristics, we could display additional information on mouseover, such as whether the sentence has more punctuation than usual. If the models are not able to work accurately enough on a sentence level, we could group sentences and employ the same process by highlighting groups of sentences. Smaller groups are preferred for usability.

![](essay_no_numbers.png)
![](wireframe1.png)
![](wireframe2.png)